<a href="https://colab.research.google.com/github/riulermendes/MBA-Distributed-Data-Processing/blob/main/trabalho.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [117]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

In [118]:
spark = SparkSession.builder.appName("Trabalho").getOrCreate()


In [119]:
import requests

csv_url = 'https://raw.githubusercontent.com/riulermendes/MBA-Distributed-Data-Processing/refs/heads/main/results_8ABDR.csv'
local_file_path = '/tmp/results_8ABDR.csv'

# Download the file
response = requests.get(csv_url)
with open(local_file_path, 'wb') as f:
    f.write(response.content)

# Read the local file with Spark
bf1 = spark.read.option("header", True) \
                .option("inferSchema", True) \
                .csv(local_file_path)

In [120]:
bf1.printSchema()
bf1.show(n=5)

root
 |-- date: date (nullable = true)
 |-- home_teamName: string (nullable = true)
 |-- away_teamName: string (nullable = true)
 |-- home_scoreHome: integer (nullable = true)
 |-- away_scoreAway: integer (nullable = true)
 |-- tournamentName: string (nullable = true)
 |-- cityCity: string (nullable = true)
 |-- countryCountry: string (nullable = true)
 |-- neutralTRUE: boolean (nullable = true)

+----------+-------------+-------------+--------------+--------------+--------------+--------+--------------+-----------+
|      date|home_teamName|away_teamName|home_scoreHome|away_scoreAway|tournamentName|cityCity|countryCountry|neutralTRUE|
+----------+-------------+-------------+--------------+--------------+--------------+--------+--------------+-----------+
|1872-11-30|     Scotland|      England|             0|             0|      Friendly| Glasgow|      Scotland|      false|
|1873-03-08|      England|     Scotland|             4|             2|      Friendly|  London|       England|   

In [121]:
bf = (
    bf1
    .withColumnRenamed('date','data')
    .withColumnRenamed('home_teamName','Mandante')
    .withColumnRenamed('away_teamName','Vistante')
    .withColumnRenamed('home_scoreHome','Placar_Mandante')
    .withColumnRenamed('away_scoreAway','Placar_Visitante')
    .withColumnRenamed('tournamentName','Torneio')
    .withColumnRenamed('cityCity','Cidade')
    .withColumnRenamed('countryCountry','País')
    .withColumnRenamed('neutralTRUE','NEUTRO')
)

In [122]:
#1 - Quantos registros existem na base?
total_records = bf.count()
print(f"Número total de registros: {total_records}")

Número total de registros: 40839


In [123]:
#2 - Quantas equipes únicas mandantes existem na base?
distinct_home_teams_count = bf.select(col("Mandante")).distinct().count()
print(f"Número de equipes mandantes únicas: {distinct_home_teams_count}")

Número de equipes mandantes únicas: 309


In [124]:
#3 - Quantas vezes as equipes mandantes saíram vitoriosas?

home_team_victories = bf.filter(
    col("Placar_Mandante") > col("Placar_Visitante")
).count()

print(f"Número de vezes que as equipes mandantes saíram vitoriosas: {home_team_victories}")

Número de vezes que as equipes mandantes saíram vitoriosas: 19864


In [125]:
#4 - Quantas vezes as equipes visitantes saíram vitoriosas?
visitor_team_victories =bf.filter(
    col("Placar_Visitante") > col("Placar_Mandante")
).count()

print(f"Número de vezes que as equipes visitantes saíram vitoriosas: {visitor_team_victories}")

Número de vezes que as equipes visitantes saíram vitoriosas: 11544


In [126]:
#5 - Quantas partidas resultaram em empate?
draw =bf.filter(
    col("Placar_Mandante") == col("Placar_Visitante")
).count()

print(f"Número de vezes que as equipes visitantes saíram vitoriosas: {draw}")

Número de vezes que as equipes visitantes saíram vitoriosas: 9431


In [127]:
#6 - Quantas partidas foram realizadas em cada país?
bf.groupBy("País").count().orderBy(col("count").desc()).show()

+--------------------+-----+
|                País|count|
+--------------------+-----+
|       United States| 1144|
|              France|  801|
|             England|  687|
|            Malaysia|  644|
|              Sweden|  637|
|             Germany|  581|
|              Brazil|  529|
|               Spain|  517|
|            Thailand|  483|
|               Italy|  480|
|         Switzerland|  477|
|             Austria|  475|
|United Arab Emirates|  472|
|        South Africa|  470|
|               Qatar|  467|
|         South Korea|  453|
|           Argentina|  449|
|             Hungary|  431|
|               Chile|  405|
|             Belgium|  396|
+--------------------+-----+
only showing top 20 rows



In [128]:
#7 - Qual país teve mais partidas?
bf.groupBy("País").count().orderBy(col("count").desc()).limit(1).show()

+-------------+-----+
|         País|count|
+-------------+-----+
|United States| 1144|
+-------------+-----+



In [129]:
#8 - Qual a partida com maior número de gols?
bf.withColumn("total_goals", col("Placar_Mandante") + col("Placar_Visitante")) \
  .orderBy(col("total_goals").desc()) \
  .select("data", "Mandante", "Vistante", "Placar_Mandante", "Placar_Visitante", "total_goals") \
  .limit(1).show()


+----------+---------+--------------+---------------+----------------+-----------+
|      data| Mandante|      Vistante|Placar_Mandante|Placar_Visitante|total_goals|
+----------+---------+--------------+---------------+----------------+-----------+
|2001-04-11|Australia|American Samoa|             31|               0|         31|
+----------+---------+--------------+---------------+----------------+-----------+



In [130]:
#9 - Qual a maior goleada?
bf.withColumn("diff", col("Placar_Mandante") - col("Placar_Visitante")) \
  .orderBy(col("diff").desc()) \
  .select("data", "Mandante", "Vistante", "Placar_Mandante", "Placar_Visitante", "diff") \
  .limit(1).show()

+----------+---------+--------------+---------------+----------------+----+
|      data| Mandante|      Vistante|Placar_Mandante|Placar_Visitante|diff|
+----------+---------+--------------+---------------+----------------+----+
|2001-04-11|Australia|American Samoa|             31|               0|  31|
+----------+---------+--------------+---------------+----------------+----+



In [131]:
#10 - Quantos jogos ocorreram no Brasil?
bf.filter(col("País") == "Brazil").count()

529